In [19]:
#!/usr/bin/env python
# -*- coding: utf-8 -*-
# @Time    : 2022/11/07 16:36
# @Author  : Wang Yujia
# @File    : PT_gen.ipynb
# @Description : Demo for calculating u_t with inferred params

# 0. What for
1. Demo for,用infer的结果来算U_t

# 1. Preparations
## 1.1 全局设置

In [86]:
# data path
data_selected_path = "../data/info_asymm/datawithnp_asc_symmetry_2_selected.csv"
# data_key_15 path
data_key_15_path = "../data/SA_PT/data_key_15.csv"
# inferred params
params_opitim_delta_wset_path = "../data/SA_PT/params_opitim_delta_wset.csv"

# output path
output_path_root = "../data/pt/results/"
# target data is from method-2
filename_head = "PT_all1288_"
filename_tail = ".csv"
filename_P = output_path_root+(filename_head + "P" + filename_tail)

features_GT = ['product_id','bidincrement','bidfee','retail']

import numpy as np
import csv
import pandas as pd
from visdom import Visdom
from SA_for_PT_funcs_delta_eq1 import *

viz = Visdom(env='P',use_incoming_socket=False)

Setting up a new session...
[Errno 11001] getaddrinfo failed
on_close() takes 1 positional argument but 3 were given
Without the incoming socket you cannot receive events from the server or register event handlers to your Visdom client.


## 1.2 data 读取
1. 保存data_key_15
2. 并且提取成功infer的data`params_all`

In [70]:
data = pd.read_csv(data_selected_path, encoding="utf-8")
params_opitim_delta_wset = pd.read_csv(params_opitim_delta_wset_path, encoding="utf-8")

idx = (pd.notna(params_opitim_delta_wset.alpha))
params_all = params_opitim_delta_wset[idx][:]
params_all.reset_index(drop=True,inplace=True)
print(f"一共有 *{params_all.shape[0]}* 个auction setting成功infer")
print(f"一共有 *{params_opitim_delta_wset.shape[0] - params_all.shape[0]}* 个auction setting未能成功infer")

data_key_15 = params_all[features_GT].copy()
data_key_15.reset_index(inplace=True)
data_key_15.to_csv(data_key_15_path,header=True, encoding="utf-8",index=False)

一共有 *1288* 个auction setting成功infer
一共有 *15* 个auction setting未能成功infer


## 1.3 functions about 'key'

In [73]:
# get key from i in 'data_key'
def get_key_from_index(i,flag="NotStr"):
    if(flag == "str"):
        key_i = list(data_key_15.iloc[i,1:])
        key_i_str = (str(int(key_i[0])),str(key_i[1]),str(key_i[2]),str(key_i[3]))

        return key_i_str
    else:
        key_i = data_key_15.iloc[i,1:]
        return key_i

#features_GT = ['product_id','bidincrement','bidfee','retail']
def select_data_fromkey(key_i_str):
    return data[(data['product_id'] == key_i_str[0]) & (data['bidincrement'] == key_i_str[1]) & (data['bidfee'] == key_i_str[2]) & (data['retail'] == key_i_str[3])].copy()

# 2. U&P
1. 得到`params_all`之后，可以对不同的auction settings做generate了
2. generate过程无非是求u-->p，u的代码在上面loss func里写过了。然后把P存到dict里
3. `P[t] = U[0]*U[1]*U[2]...*U[t]*U[t+1]`
4. `P[key_i_str][max_T]==tmp[max_T]*U[max_T]`

In [83]:
# solve for U&P respectively for all settings
P = {}
# params_all.shape[0]
for i in range(0,params_all.shape[0]):

    # get optimized params
    alpha, labda = params_all.alpha[i],params_all.labda[i]

    # get i_th data_key
    key_i = get_key_from_index(i)
    data_i = select_data_fromkey(key_i)
    data_i.reset_index(drop=True,inplace=True)

    # Get params
    T_i = data_i['N'].astype(int)                   # auction duration sequence
    max_T = int(max(T_i))                           # max duration value
    cnt_n_2_i = data_i['cnt_n_2'].astype(int)       # Number of occurrences of different durations
    v = float(data_i['retail'].unique())            # retail price = valuation
    d = float(data_i['bidincrement'].unique())      # bid increment
    b = float(data_i['bidfee'].unique())            # bid fee

    U= [0] * (max_T + 1)                            # the prob. that someone offers a bid in t_th round
    U[0] = 1
    key_i_str = get_key_from_index(i,"str")         # key for P
    # print(f"in {i}_th, the key_i_str is: \n {key_i_str}")

    P[key_i_str] = np.array([0.0]*(max_T+1))
    P_tmp = [0.0]*(max_T+1)                         # P is what we want to generate
    P_tmp[0] = 1
    tmp = np.array([0.0]*(max_T+2))                 # tmp的大小其实不需要太精确
    tmp[0] = 1.0

    # solve for U
    for t in range(1,max_T+1):
        U[t] = f_Equi(t, v, d, b, alpha, labda)

    # solve for P
    for t in range(0,max_T):
        tmp[t+1] = tmp[t]*U[t]                      # tmp[t]存了到(t-1)的连乘积
        P[key_i_str][t] = (1-U[t+1])*tmp[t+1]

    P[key_i_str][max_T] = tmp[max_T]*U[max_T]
    # assert(P[key_i_str][max_T]==tmp[max_T]*U[max_T],"Wrong")

    # if(i%10 == 0):
    viz.line(P[key_i_str],np.arange(0,P[key_i_str].shape[0]),win = 'P_'+str(i),opts= dict(title = f'P_{i}_v={v}_b={b}_d={d}'))

[Errno 11001] getaddrinfo failed
on_close() takes 1 positional argument but 3 were given
[Errno 11001] getaddrinfo failed
on_close() takes 1 positional argument but 3 were given
[Errno 11001] getaddrinfo failed
on_close() takes 1 positional argument but 3 were given
[Errno 11001] getaddrinfo failed
on_close() takes 1 positional argument but 3 were given
[Errno 11001] getaddrinfo failed
on_close() takes 1 positional argument but 3 were given
[Errno 11001] getaddrinfo failed
on_close() takes 1 positional argument but 3 were given
[Errno 11001] getaddrinfo failed
on_close() takes 1 positional argument but 3 were given
[Errno 11001] getaddrinfo failed
on_close() takes 1 positional argument but 3 were given
[Errno 11001] getaddrinfo failed
on_close() takes 1 positional argument but 3 were given
[Errno 11001] getaddrinfo failed
on_close() takes 1 positional argument but 3 were given
[Errno 11001] getaddrinfo failed
on_close() takes 1 positional argument but 3 were given
[Errno 11001] getaddr

## 2.1 save

In [87]:
with open(filename_P,"w+",encoding='utf—8') as f:
    w=csv.writer(f)        # 每行存储一个键值对
    w.writerow(P.items())

[Errno 11001] getaddrinfo failed
on_close() takes 1 positional argument but 3 were given
